<a href="https://colab.research.google.com/github/shubham3032002/CrewAi/blob/main/crewai.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip -q install crewai crewai_tools langchain langchain_community langchain_groq

In [23]:
!pip -q install langchain-groq crewai python-dotenv serpapi

In [24]:
pip install --upgrade crewai

ERROR:root:Failed to get supported params: argument of type 'NoneType' is not iterable
ERROR:root:Failed to get supported params: argument of type 'NoneType' is not iterable
ERROR:root:Failed to get supported params: argument of type 'NoneType' is not iterable
ERROR:root:Failed to get supported params: argument of type 'NoneType' is not iterable
ERROR:root:Failed to get supported params: argument of type 'NoneType' is not iterable
ERROR:root:Failed to get supported params: argument of type 'NoneType' is not iterable
ERROR:root:Failed to get supported params: argument of type 'NoneType' is not iterable
ERROR:root:Failed to get supported params: argument of type 'NoneType' is not iterable
ERROR:root:LiteLLM call failed: litellm.BadRequestError: LLM Provider NOT provided. Pass in the LLM provider you are trying to call. You passed model=llama3-70b-8192
 Pass model as E.g. For 'Huggingface' inference endpoints pass in `completion(model='huggingface/starcoder',..)` Learn more: https://docs.


Provider List: https://docs.litellm.ai/docs/providers


Provider List: https://docs.litellm.ai/docs/providers


Provider List: https://docs.litellm.ai/docs/providers


Provider List: https://docs.litellm.ai/docs/providers


Provider List: https://docs.litellm.ai/docs/providers


Provider List: https://docs.litellm.ai/docs/providers

 
[2025-02-24 18:48:41][INFO]: Planning the crew execution

Provider List: https://docs.litellm.ai/docs/providers


Provider List: https://docs.litellm.ai/docs/providers


Provider List: https://docs.litellm.ai/docs/providers

 Error during LLM call: litellm.BadRequestError: LLM Provider NOT provided. Pass in the LLM provider you are trying to call. You passed model=llama3-70b-8192
 Pass model as E.g. For 'Huggingface' inference endpoints pass in `completion(model='huggingface/starcoder',..)` Learn more: https://docs.litellm.ai/docs/providers


BadRequestError: litellm.BadRequestError: LLM Provider NOT provided. Pass in the LLM provider you are trying to call. You passed model=llama3-70b-8192
 Pass model as E.g. For 'Huggingface' inference endpoints pass in `completion(model='huggingface/starcoder',..)` Learn more: https://docs.litellm.ai/docs/providers

In [34]:
from langchain_groq.chat_models import ChatGroq
from langchain.tools import Tool
from langchain_community.utilities import SerpAPIWrapper
from crewai import Agent, Task, Crew
from tenacity import retry, stop_after_attempt, wait_exponential
import os
import time


SERPAPI_KEY = "7******************3"
GROQ_API_KEY = "g****************************2"

if not all([SERPAPI_KEY, GROQ_API_KEY]):
    raise ValueError("Missing required API keys")

os.environ["SERPAPI_API_KEY"] = SERPAPI_KEY
os.environ["GROQ_API_KEY"] = GROQ_API_KEY


llm = ChatGroq(
    model_name="groq/mixtral-8x7b-32768",
    temperature=0.7,
    max_tokens=400,
    request_timeout=30
)


class SafeSearch:
    def __init__(self):
        self.search = SerpAPIWrapper()

    def run(self, query: str) -> str:
        """Simple string input search"""
        try:
            return self.search.run(query)
        except Exception as e:
            return f"Search unavailable. Error: {str(e)}"

search_tool = Tool(
    name="WebSearch",
    func=SafeSearch().run,
    description="Search web for current information"
)

researcher = Agent(
    role="AI Analyst",
    goal="Identify key AI trends for 2024",
    backstory="Expert in technology trend analysis",
    tools=[search_tool],
    verbose=True,
    llm=llm,
    max_iter=2,
    step_callback=lambda x: time.sleep(5)
)

writer = Agent(
    role="Technical Writer",
    goal="Create clear technical content",
    backstory="Experienced technology communicator",
    verbose=True,
    llm=llm,
    max_iter=2
)


research_task = Task(
    description="Identify key AI trends for 2024 using simple search queries",
    expected_output="3-paragraph report with verified sources",
    agent=researcher,
    config={'allow_delegation': False}
)

write_task = Task(
    description="Create blog post with markdown formatting",
    expected_output="4-paragraph markdown with headings",
    agent=writer,
    output_file="ai_trends.md"
)


crew = Crew(
    agents=[researcher, writer],
    tasks=[research_task, write_task],
    verbose=True,
    process="sequential",
    memory=False
)


@retry(stop=stop_after_attempt(2),
       wait=wait_exponential(multiplier=2, min=5, max=60))
def execute_safely():
    try:
        return crew.kickoff()
    except Exception as e:
        print(f"Pausing due to error: {str(e)}")
        time.sleep(30)
        raise

if __name__ == "__main__":
    print("Starting optimized workflow...")
    try:
        result = execute_safely()
        print("\nSuccessful Output:\n", result)
    except Exception as e:
        print("\nWorkflow failed. Final error:", str(e))
        print("Recommendations:")
        print("- Verify API keys are valid and active")
        print("- Check service quotas at:")
        print("  https://serpapi.com/manage-api-key")
        print("  https://console.groq.com/quotas")

Starting optimized workflow...
# Agent: AI Analyst
## Task: Identify key AI trends for 2024 using simple search queries
 

I encountered an error while trying to use the tool. This was the error: Arguments validation failed: 1 validation error for WebSearchInput
query
  Input should be a valid string [type=string_type, input_value={'description': 'AI trends 2023', 'type': 'str'}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.10/v/string_type.
 Tool WebSearch accepts these inputs: Tool Name: WebSearch
Tool Arguments: {'query': {'description': '', 'type': 'str'}}
Tool Description: Search web for current information



# Agent: AI Analyst
## Thought: To identify key AI trends for 2024, I should first search for recent articles and reports predicting the future of AI. I will start by searching for "AI trends 2023" to get a sense of the current predictions for the near future, and then look for information on how these trends might evolve by 2024.
## Using